In [2]:
import pandas as pd

col_list = ["countyFIPS", "population"]
county_population = pd.read_csv("covid_county_population_usafacts.csv", usecols=col_list)

print(county_population)

      countyFIPS  population
0              0           0
1           1001       55869
2           1003      223234
3           1005       24686
4           1007       22394
...          ...         ...
3190       56037       42343
3191       56039       23464
3192       56041       20226
3193       56043        7805
3194       56045        6927

[3195 rows x 2 columns]


In [3]:
col_list2 = ["countyFIPS", "cumul_confirmed"]
county_confirmed_cases = pd.read_csv("covid_confirmed_usafacts.csv", usecols = col_list2)

county_confirmed = county_confirmed_cases["cumul_confirmed"]

print(county_confirmed_cases)

      countyFIPS  cumul_confirmed
0              0                0
1           1001             2230
2           1003             7097
3           1005             1080
4           1007              907
...          ...              ...
3190       56037              658
3191       56039              897
3192       56041              529
3193       56043              179
3194       56045              203

[3195 rows x 2 columns]


In [7]:
col_list3 = ["countyFIPS", "cumul_death"]
county_death_cases = pd.read_csv("covid_deaths_usafacts.csv", usecols = col_list3)

county_death = county_death_cases["cumul_death"]
print(county_death_cases)

      countyFIPS  cumul_death
0              0            0
1           1001           30
2           1003           77
3           1005            9
4           1007           15
...          ...          ...
3190       56037            4
3191       56039            2
3192       56041            3
3193       56043            7
3194       56045            0

[3195 rows x 2 columns]


In [8]:
result = pd.concat([county_population, county_confirmed, county_death], axis=1)

print(result)

      countyFIPS  population  cumul_confirmed  cumul_death
0              0           0                0            0
1           1001       55869             2230           30
2           1003      223234             7097           77
3           1005       24686             1080            9
4           1007       22394              907           15
...          ...         ...              ...          ...
3190       56037       42343              658            4
3191       56039       23464              897            2
3192       56041       20226              529            3
3193       56043        7805              179            7
3194       56045        6927              203            0

[3195 rows x 4 columns]
